In [16]:
df = pd.read_csv('modis_2022_Brazil.csv')
df

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,-23.1626,-53.7109,304.9,2.6,1.5,2022-01-01,244,Terra,MODIS,62,6.03,290.3,26.2,N,0
1,-23.3932,-53.8741,306.2,2.5,1.5,2022-01-01,244,Terra,MODIS,67,6.03,290.8,25.4,N,0
2,-23.3858,-53.8703,305.4,2.5,1.5,2022-01-01,244,Terra,MODIS,59,6.03,290.7,21.6,N,0
3,-20.2370,-56.6381,311.9,1.8,1.3,2022-01-01,245,Terra,MODIS,84,6.03,293.7,23.2,N,0
4,-20.2169,-56.6297,310.5,1.8,1.3,2022-01-01,245,Terra,MODIS,80,6.03,293.8,19.5,N,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362780,4.2887,-60.6617,322.9,1.3,1.1,2022-12-31,1730,Aqua,MODIS,77,6.03,298.2,14.4,D,0
362781,4.0736,-61.0059,323.0,1.4,1.2,2022-12-31,1730,Aqua,MODIS,77,6.03,297.5,19.0,D,0
362782,3.8829,-59.8814,323.9,1.2,1.1,2022-12-31,1730,Aqua,MODIS,69,6.03,298.4,11.2,D,0
362783,3.8733,-59.8800,320.2,1.2,1.1,2022-12-31,1730,Aqua,MODIS,29,6.03,297.8,6.3,D,0


In [ ]:
average_confidence = df_filtered.groupby(['latitude', 'longitude'])['confidence'].mean().reset_index(name='avg_confidence')

# If you want to merge this average confidence back into the original DataFrame,
# so that each row has the average 'confidence' for its location:
df_merged = pd.merge(df_filtered, average_confidence, how='left', on=['latitude', 'longitude'])

In [94]:
import pandas as pd
import numpy as np
from datetime import datetime
import requests

# Function to calculate distance between two points
def haversine(lat1, lon1, lat2, lon2):
    # Radius of the Earth in km
    R = 6371.0
    # Conversion to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    # Differences in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    # Haversine formula
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = R * c
    return distance



# Load the CSV file
df = pd.read_csv('modis_2022_Brazil.csv')

# Filter data points within a 50km radius of the specified coordinates
origin_lat, origin_lon = -23.162, -53.7109
df['distance'] = haversine(origin_lat, origin_lon, df['latitude'], df['longitude'])
df_filtered = df[df['distance'] <= 10000000000000000].copy()
from scipy.spatial import KDTree

from sklearn.cluster import DBSCAN
import numpy as np
import pandas as pd

# Convert lat/lon to radians for use in Haversine metric
coords = df_filtered[['latitude', 'longitude']].to_numpy()
coords_rad = np.radians(coords)

# DBSCAN clustering
kms_per_radian = 6371.0088  # Mean radius of the earth
epsilon = 1 / kms_per_radian  # 1km search radius
db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(coords_rad)

# Assign cluster labels to the DataFrame
df_filtered['cluster'] = db.labels_

# Calculate the average confidence for each cluster
df_filtered['avg_confidence'] = df_filtered.groupby('cluster')['confidence'].transform('mean')



In [95]:
# multiply avg confidence by number of points in cluster
df_filtered['weighted_avg_confidence'] = df_filtered['avg_confidence'] * df_filtered.groupby('cluster')['confidence'].transform('count')

# get number of entries per clsuter
df_filtered['count'] = df_filtered.groupby('cluster')['confidence'].transform('count')
df_filtered

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,distance,cluster,avg_confidence,weighted_avg_confidence,count
0,-23.1626,-53.7109,304.9,2.6,1.5,2022-01-01,244,Terra,MODIS,62,6.03,290.3,26.2,N,0,0.066717,0,69.312500,2218.0,32
1,-23.3932,-53.8741,306.2,2.5,1.5,2022-01-01,244,Terra,MODIS,67,6.03,290.8,25.4,N,0,30.639822,1,24.100000,241.0,10
2,-23.3858,-53.8703,305.4,2.5,1.5,2022-01-01,244,Terra,MODIS,59,6.03,290.7,21.6,N,0,29.738743,1,24.100000,241.0,10
3,-20.2370,-56.6381,311.9,1.8,1.3,2022-01-01,245,Terra,MODIS,84,6.03,293.7,23.2,N,0,444.086473,2,64.896552,1882.0,29
4,-20.2169,-56.6297,310.5,1.8,1.3,2022-01-01,245,Terra,MODIS,80,6.03,293.8,19.5,N,0,445.151640,2,64.896552,1882.0,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362780,4.2887,-60.6617,322.9,1.3,1.1,2022-12-31,1730,Aqua,MODIS,77,6.03,298.2,14.4,D,0,3144.232332,151457,77.000000,77.0,1
362781,4.0736,-61.0059,323.0,1.4,1.2,2022-12-31,1730,Aqua,MODIS,77,6.03,297.5,19.0,D,0,3130.225404,151458,77.000000,77.0,1
362782,3.8829,-59.8814,323.9,1.2,1.1,2022-12-31,1730,Aqua,MODIS,69,6.03,298.4,11.2,D,0,3080.887443,151459,69.000000,69.0,1
362783,3.8733,-59.8800,320.2,1.2,1.1,2022-12-31,1730,Aqua,MODIS,29,6.03,297.8,6.3,D,0,3079.811351,151460,29.000000,29.0,1


In [96]:
df_filtered = df_filtered.drop_duplicates(subset='cluster')
# keep only best 25% of rows with highest weighted average confidence
df_filtered = df_filtered.nlargest(int(len(df_filtered)*0.001), 'weighted_avg_confidence')

In [97]:
df_filtered

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,distance,cluster,avg_confidence,weighted_avg_confidence,count
76114,-7.7788,-55.0073,320.1,1.4,1.2,2022-07-30,1739,Aqua,MODIS,71,6.03,298.4,14.1,D,0,1716.124854,34973,76.532207,72476.0,947
72049,-9.1773,-59.7073,307.1,1.0,1.0,2022-07-28,535,Aqua,MODIS,70,6.03,293.8,7.7,N,0,1680.961116,33575,70.203562,55180.0,786
96585,-8.8077,-62.4087,317.0,1.1,1.0,2022-08-13,1748,Aqua,MODIS,47,6.03,302.4,7.8,D,0,1845.613204,42400,72.225710,48319.0,669
10930,-10.8526,-53.9489,311.4,1.8,1.3,2022-03-02,1658,Aqua,MODIS,56,6.03,293.0,14.0,D,0,1368.975610,7518,73.327243,44143.0,602
20097,-11.3154,-53.9979,325.8,1.2,1.1,2022-04-29,1730,Aqua,MODIS,79,6.03,302.4,20.8,D,0,1317.632767,12531,74.901235,42469.0,567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73762,-7.5255,-60.7774,384.2,1.0,1.0,2022-07-29,1412,Terra,MODIS,100,6.03,308.9,169.5,D,0,1895.476714,34117,71.163934,8682.0,122
135825,-12.9662,-53.1497,351.3,1.1,1.0,2022-08-24,1725,Aqua,MODIS,95,6.03,308.0,61.5,D,0,1135.267032,54029,71.454545,8646.0,121
117405,-9.1674,-65.8887,312.1,1.5,1.2,2022-08-20,1752,Aqua,MODIS,30,6.03,296.4,10.0,D,0,2025.334196,49147,70.540984,8606.0,122
276303,-11.2983,-43.4274,353.0,1.0,1.0,2022-10-01,1639,Aqua,MODIS,93,6.03,313.7,48.6,D,0,1710.925736,105566,80.392523,8602.0,107


In [58]:
# get one of datapoint from each cluster with confidence > 80
df_filtered = df_filtered[df_filtered['avg_confidence'] > 85].drop_duplicates(subset='cluster')
df_filtered

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,distance,cluster,avg_confidence
371,-29.5893,-56.6496,337.1,3.6,1.8,2022-01-04,1700,Aqua,MODIS,88,6.03,294.4,170.0,D,0,772.214896,111,86.5
1048,2.3078,-61.6232,347.2,1.4,1.2,2022-01-09,1727,Aqua,MODIS,94,6.03,295.0,78.7,D,0,2959.033723,269,94.5
1298,-29.6172,-56.1091,332.3,2.6,1.5,2022-01-11,1706,Aqua,MODIS,72,6.03,312.8,49.1,D,0,756.420838,319,89.0
1882,-21.3492,-53.3695,335.7,1.4,1.2,2022-01-14,1739,Aqua,MODIS,84,6.03,298.7,37.8,D,0,204.612756,447,86.0
2128,-13.8367,-39.5604,332.4,1.0,1.0,2022-01-16,1253,Terra,MODIS,86,6.03,293.0,27.9,D,0,1814.945534,499,86.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357284,-9.9039,-37.9256,339.8,1.3,1.1,2022-12-13,1607,Aqua,MODIS,88,6.03,307.2,43.2,D,0,2233.221819,4970,92.5
357288,-6.4936,-40.1785,331.1,1.6,1.3,2022-12-13,1608,Aqua,MODIS,82,6.03,289.3,49.3,D,0,2352.107174,4971,90.5
360272,-5.7199,-38.5829,340.2,2.4,1.5,2022-12-19,1310,Terra,MODIS,89,6.03,296.5,123.1,D,0,2527.764300,5013,89.0
361875,-21.7261,-57.4484,357.6,1.0,1.0,2022-12-27,1349,Terra,MODIS,98,6.03,309.9,64.8,D,0,415.950643,5061,93.0


In [98]:
# put lat lon in dict
lat_lon_dict = df_filtered[['latitude', 'longitude']].to_dict(orient='records')

# save to json
import json
with open('output.json', 'w') as f:
    json.dump(lat_lon_dict, f)
    